In [1]:
import os
import re
from CustomFunctions import image_processing

In [2]:
################ LOAD THE DATA FOR THE RANDOM CELLS #############

croppeddir = '//10.158.28.35/TheriotLab_LLS7/Aaron/manual_crop_lls_decon/'
another = os.listdir(croppeddir)
dirr = 'E:/Aaron/random_lls_smooth/'
savedir = dirr + 'processed_images/'
posdir = dirr + 'position_info/'
if not os.path.exists(savedir):
    os.makedirs(savedir)
if not os.path.exists(posdir):
    os.makedirs(posdir)

decon = True
orig_size = False
xy_buffer = 25
z_buffer = 25
hilo = True

#get all unique cells that I followed on the scope
cell_list = list(set([re.findall(r'(.*?_cell\d)', x)[0] for x in another]))
#exclude dates that had funny things happen
cell_list = [x for x in cell_list if '20240528' not in x]
#exclude drug data
cell_list = [x for x in cell_list if ('DMSO' not in x) and ('CK666' not in x) and ('Para' not in x)]
#sort the list so that it happens in the same order every time
cell_list.sort()

In [3]:
################ SEGMENT AND TRACK CELLS #############
if __name__ ==  '__main__':
    for cellstr in cell_list:
        image_processing.segment_and_crop_LLS_manual(
                cellstr,#the name of the unique cell being cropped and segmented across multiple videos
                savedir, #direcetory to save images
                posdir, #directory to save position data
                croppeddir, #directory where all of the cropped LLS images live
                decon, #are these images deconvolved?
                orig_size, #should we save the images at their original size?
                xy_buffer, #crop buffer in x-y
                z_buffer, #crrop buffer in z
                hilo, #whether or not to do multiple thresholds for segmenting secondary signals
                )

Finished segmenting cropped images of 20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2-01-Subset-01_deskewed_decon.czi
Finished segmenting cropped images of 20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2-02-Subset-01_deskewed_decon.czi
Finished segmenting cropped images of 20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2-03-Subset-01_deskewed_decon.czi
Finished segmenting cropped images of 20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2-04-Subset-01_deskewed_decon.czi
Finished segmenting cropped images of 20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2-05-Subset-01_deskewed_decon.czi
Finished segmenting cropped images of 20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2-06-Subset-01_deskewed_decon.czi
Finished segmenting cropped images of 20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2-07-Subset-01_deskewed_decon.czi
Finished segmenting cropped images of 20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2-08-Subset-01_deskewed_decon.czi
Finished segmenting cropped imag

Finished segmenting cropped images of 20240523_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2-14-Subset-01_deskewed_decon.czi
Finished segmenting cropped images of 20240523_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2-15-Subset-01_deskewed_decon.czi
Finished segmenting cropped images of 20240523_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2-16-Subset-01_deskewed_decon.czi
Finished segmenting cropped images of 20240523_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2-17-Subset-01_deskewed_decon.czi
Finished segmenting cropped images of 20240523_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2-18-Subset-01_deskewed_decon.czi
Finished segmenting cropped images of 20240523_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2-19-Subset-01_deskewed_decon.czi
Finished segmenting cropped images of 20240523_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2-03-Subset-02_deskewed_decon.czi
Finished segmenting cropped images of 20240523_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2-04-Subset-03_deskewed_decon.czi
Finished segmenting cropped imag

Finished segmenting cropped images of 20240524_488_EGFP-CAAX_640_actin-halotag_cell2-07-Subset-03_deskewed_decon.czi
Finished segmenting cropped images of 20240524_488_EGFP-CAAX_640_actin-halotag_cell2-08-Subset-03_deskewed_decon.czi
Finished segmenting cropped images of 20240524_488_EGFP-CAAX_640_actin-halotag_cell2-08-Subset-04_deskewed_decon.czi
Finished segmenting cropped images of 20240524_488_EGFP-CAAX_640_actin-halotag_cell2-09-Subset-04_deskewed_decon.czi
Finished segmenting cropped images of 20240524_488_EGFP-CAAX_640_actin-halotag_cell2-10-Subset-04_deskewed_decon.czi
Finished segmenting cropped images of 20240524_488_EGFP-CAAX_640_actin-halotag_cell2-09-Subset-05_deskewed_decon.czi
Finished segmenting cropped images of 20240524_488_EGFP-CAAX_640_actin-halotag_cell2-12-Subset-06_deskewed_decon.czi
Finished segmenting cropped images of 20240524_488_EGFP-CAAX_640_actin-halotag_cell3-01-Subset-01_deskewed_decon.czi
Finished segmenting cropped images of 20240524_488_EGFP-CAAX_640

Finished segmenting cropped images of 20240611_488_EGFP-CAAX_640_actin-halotag_cell2-05-Subset-02_deskewed_decon.czi
Finished segmenting cropped images of 20240611_488_EGFP-CAAX_640_actin-halotag_cell2-06-Subset-02_deskewed_decon.czi
Finished segmenting cropped images of 20240611_488_EGFP-CAAX_640_actin-halotag_cell2-07-Subset-02_deskewed_decon.czi
Finished segmenting cropped images of 20240611_488_EGFP-CAAX_640_actin-halotag_cell2-01-Subset-03_deskewed_decon.czi
Finished segmenting cropped images of 20240611_488_EGFP-CAAX_640_actin-halotag_cell2-07-Subset-03_deskewed_decon.czi
Finished segmenting cropped images of 20240611_488_EGFP-CAAX_640_actin-halotag_cell2-09-Subset-03_deskewed_decon.czi
Finished segmenting cropped images of 20240611_488_EGFP-CAAX_640_actin-halotag_cell2-10-Subset-03_deskewed_decon.czi
Finished segmenting cropped images of 20240611_488_EGFP-CAAX_640_actin-halotag_cell2-11-Subset-03_deskewed_decon.czi
Finished segmenting cropped images of 20240611_488_EGFP-CAAX_640

Finished segmenting cropped images of 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell2-07-Subset-01_deskewed_decon.czi
Finished segmenting cropped images of 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell2-08-Subset-01_deskewed_decon.czi
Finished segmenting cropped images of 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell2-09-Subset-01_deskewed_decon.czi
Finished segmenting cropped images of 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell2-10-Subset-01_deskewed_decon.czi
Finished segmenting cropped images of 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell2-11-Subset-01_deskewed_decon.czi
Finished segmenting cropped images of 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell2-12-Subset-01_deskewed_decon.czi
Finished segmenting cropped images of 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell2-13-Subset-01_deskewed_decon.czi
Finished segmenting cropped images of 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell2-14-Subset-01_deskewed_decon.czi
Finished segmenting cropped images of 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell2-01-Sub

In [25]:
df = cellinfo[cellinfo.CellID == cellinfo.CellID.unique()[0]]

In [45]:
###### Get the trajectory info from all of the position info

#define directory stuff
dirr = 'E:/Aaron/random_lls/'
savedir = dirr + 'processed_images/'
datadir = dirr + 'processed_data/'
if not os.path.exists(savedir):
    os.makedirs(savedir)

time_interval = 5
    
# #combine all of the cell csvs into one dataframe
# fileslist = [x for x in os.listdir(datadir) if x.endswith('.csv')]
# if __name__ == '__main__':
#     pool = multiprocessing.Pool(processes=60)
#     csvlist = [datadir+i for i in fileslist]
#     celllist = pool.map(multicsv, csvlist)
#     pool.close()
#     pool.join()
# cellinfo = pd.concat(celllist).reset_index(drop=True)
# cellinfo['CellID'] = [re.split(r'(-\d+-Subset)',x)[0]+ '_'+ re.findall(r'Subset-(\d+)', x)[0] for x in cellinfo.cell.to_list()]

dflist = []
# for i, df in cellinfo.groupby('CellID'):
df = cellinfo[cellinfo.CellID == cellinfo.CellID.unique()[0]]
#first get dataframe in time order
df = df.sort_values(by = 'time').reset_index(drop=True)


#make sure there are no gaps due to failed segmentations
if any(abs(df.frame.diff())>1):
    diff = df.time.diff()
    difflist = [0]
    difflist.extend(diff[diff>5].index.to_list())
    runs = []
    for x in range(len(difflist)-1):
        runs.append(list(range(difflist[x], difflist[x+1])))
else:
    runs = [df.index.to_list()]

#save the df in case it gets broken up later    
brokendf = df.copy()

for r in runs:
    if len(r)>2:
        df = brokendf.iloc[r].reset_index(drop=True)
        #set the k order for interpolation to the max possible
        if len(df)<6:
            kay = len(df)-1
        else:
            kay = 5

        pos = df[['x','y','z']]
        if bool(pos[pos.duplicated()].index.tolist()):
            ######### FIND CELL TRAJECTORY AND EULER ANGLES ################
            # if there is duplicate positions
            dups = pos[pos.duplicated()].index.tolist()
            pos_drop = pos.drop(dups, axis = 0)
            if pos_drop.shape[0]<3:
                traj = np.zeros([1,len(pos),3])
                trajsmo = pos.to_numpy().copy()
            else:
                #get trajectories without the duplicates
                tck, u = interpolate.splprep(pos_drop.to_numpy().T, k=kay, s=5)
                yderv = interpolate.splev(u,tck,der=1)
                traj = np.vstack(yderv).T
                #get smoothened trajectory
                ysmo = interpolate.splev(u,tck,der=0)
                trajsmo = np.vstack(ysmo).T
                #re-insert duplicate row that was dropped
                for d, dd in enumerate(dups):
                    traj = np.insert(traj, dd, traj[dd-1,:], axis=0)
                    trajsmo = np.insert(trajsmo, dd, trajsmo[dd-1,:], axis=0)

        else:
            ######### FIND CELL TRAJECTORY AND EULER ANGLES ################
            #no duplicate positions
            #interpolate and get tangent at midpoint
            tck, b = interpolate.splprep(pos.to_numpy().T, k=kay, s=5)
            yderv = interpolate.splev(b,tck,der=1)
            traj = np.vstack(yderv).T
            #get smoothened trajectory
            ysmo = interpolate.splev(b,tck,der=0)
            trajsmo = np.vstack(ysmo).T

        ###add smoothened trajectory positions 
        #change x y z names in the dataframe
        df.rename(columns={"x": "x_raw", "y": "y_raw", "z": "z_raw"}, inplace = True)
        #add smoothened positions
        df['x'] = trajsmo[:,0]
        df['y'] = trajsmo[:,1]
        df['z'] = trajsmo[:,2]

        ############## Bayesian persistence and activity #################
        persistence, activity, speed = get_pa(df, time_interval)
        df['persistence'] = np.concatenate([np.array([np.nan]*2), persistence])
        df['activity'] = np.concatenate([np.array([np.nan]*2), activity])
        df['speed'] = np.concatenate([np.array([np.nan]), speed])
        df['avg_persistence'] = np.array([persistence.mean()]*(len(persistence)+2))
        df['avg_activity'] = np.array([activity.mean()]*(len(activity)+2))
        df['avg_speed'] = np.array([speed.mean()]*(len(speed)+1))

        #add directional autocorrelations
        df['directional_autocorrelation'] = DA_3D(df[['x','y','z']].to_numpy())

        #get the trajectory and the previous trajectory for each frame and 
        #save as an individual dataframe for each cell and frame
        for v, row in df.iterrows():
            if v==0:
                row['Prev_Trajectory_X'] = np.nan
                row['Prev_Trajectory_Y'] = np.nan
                row['Prev_Trajectory_Z'] = np.nan
                row['Trajectory_X'] = traj[v,0]
                row['Trajectory_Y'] = traj[v,1]
                row['Trajectory_Z'] = traj[v,2]
                row['Turn_Angle'] = np.nan
                dflist.append(row)

            if v>0:
                row['Prev_Trajectory_X'] = traj[v-1,0]
                row['Prev_Trajectory_Y'] = traj[v-1,1]
                row['Prev_Trajectory_Z'] = traj[v-1,2]
                row['Trajectory_X'] = traj[v,0]
                row['Trajectory_Y'] = traj[v,1]
                row['Trajectory_Z'] = traj[v,2]
                if all(traj[v-1,:] == traj[v,:]):
                    row['Turn_Angle'] = 0
                else:
                    row['Turn_Angle'] = angle_distance(traj[v-1,0], traj[v-1,1], traj[v-1,2], traj[v,0], traj[v,1], traj[v,2])
                dflist.append(row)


In [48]:
df = cellinfo[cellinfo.CellID == cellinfo.CellID.unique()[0]]
#first get dataframe in time order
df = df.sort_values(by = 'time').reset_index(drop=True)
df.iloc[0].speed

nan

In [4]:
############## find the width rotation angles through time

croppeddir = '//10.158.28.35/TheriotLab_LLS7/Aaron/manual_crop_lls_decon/'
another = os.listdir(croppeddir)
dirr = 'E:/Aaron/random_lls/'
imdir = dirr + 'processed_images/'
csvdir = dirr + 'processed_data/'
savedir = dirr + 'Data_and_Figs/'
if not os.path.exists(savedir):
    os.makedirs(savedir)

    

xyres = 0.145 #um / pixel 
zstep = 0.145 # um
align_method = 'trajectory'
sigma = 0
allresults = []

    
#get list of frames and cells that were actually fully analyzed
datalist = [x.split('_cell_info.csv')[0] for x in os.listdir(csvdir)]
#get all of the names of the cells that were followed
imlist = list(set([re.findall(r'(.*?_cell\d)', x)[0] for x in datalist]))
imlist.sort()
#get only the segmented images from the imdir
seglist = [x for x in os.listdir(imdir) if 'segmented' in x]
for i in imlist:
    #get all of the images from a particular cell I was following as long as they were analyzed
    curimlist = [x for x in seglist if i in x and x.split('_segmented.tiff')[0] in datalist]
    #find the total number of cells I cropped while following the cell of interest
    cellnums = list(set([re.findall(r'Subset-(\d+)',x)[0] for x in curimlist]))
    cellnums.sort()
    for cn in cellnums:
        #get all the images of a given cell
        curcell = [x for x in curimlist if f'Subset-{cn}' in x]
        #sort the current cell to be in chronological order
        curcell.sort(key=lambda x: float(re.findall(r'(\d+)-Subset', x)[0]))
        if __name__ ==  '__main__':
            results = []
            pool = multiprocessing.Pool(processes=60)
            for c in curcell:
                #get path to segmented image
                impath = imdir + c

                #put in the pool
                pool.apply_async(shparam_mod.find_normal_width_peaks, args = (
                    impath,
                    csvdir,
                    xyres,
                    zstep,
                    sigma,
                    align_method,
                    ),             
                    callback = collect_results)

            pool.close()
            pool.join()
            
        #sort the results based on the movie order and frame order
        results.sort(key=lambda x: (float(re.findall(r'(\d+)-Subset', x[0])[0]), float(re.findall('(?<=frame_)\d*', x[0])[0])))
        #put results into an ordered dataframe
        tempframe = pd.DataFrame(results, columns = ['cell','Width_Peaks'])
        tempframe['frame'] = [float(re.findall('(?<=frame_)\d*', x[0])[0]) for x in results]
    
        runs = list()
        #######https://stackoverflow.com/questions/2361945/detecting-consecutive-integers-in-a-list
        for k, g in groupby(enumerate(tempframe['frame']), lambda ix: ix[0] - ix[1]):
            currentrun = list(map(itemgetter(1), g))
            list.append(runs, currentrun)


        #find the minima in each frame that are closest to the minimum chosen in the last frame
        #aka the one that results in the least amount of consecutive rotation
        fullminlist = []
        runcount = 0
        for xx in runs:
            runframe = tempframe.iloc[runcount:int(runcount+len(xx))]
            #add to runcount
            runcount = runcount + len(xx)
            wplist = runframe.Width_Peaks.to_list()
            seeds = []
            allallmins = []
            #for all the starting peaks find the least different rotations through time
            for s in wplist[0]:
                allmins = [s]
                for wp in wplist[1:]:
                    if bool(len(wp) == 0):
                        allmins.append(allmins[-1])
                    else:
                        allmins.append(wp[np.argmin(abs(wp-(allmins[-1])))])
                allallmins.append(allmins)
                seeds.append(np.sum(abs(np.diff(allmins))))
            #add rotations of current run to the list
            fullminlist.extend(allallmins[np.argmin(seeds)])


        #add all mins to tempframe
        tempframe['Closest_minimums'] = fullminlist

        allresults.append(tempframe)

        print(f'Finished {i} {cn}')

#save the shape metrics dataframe
bigdf = pd.concat(allresults)
bigdf.to_csv(savedir + 'Closest_Width_Peaks.csv')


Finished 20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2 01
Finished 20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2 02
Finished 20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2 03
Finished 20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2 04
Finished 20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2 05
Finished 20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2 06
Finished 20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2 07
Finished 20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2 08
Finished 20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_cell2 09
Finished 20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_cell3 01
Finished 20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_cell3 02
Finished 20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_cell3 03
Finished 20240520_488_EGFP-CAAX_561_mysoin-mApple_37C_cell3 04
Finished 20240523_488_EGFP-CAAX_561_mysoin-mApple_37C_cell1 01
Finished 20240523_488_EGFP-CAAX_561_mysoin-mApple_37C_cell1 02
Finished 20240523_488_EGFP-CAAX_561_mysoin-mApple_37C_c

Finished 20240626_488_EGFP-CAAX_640_SPY650-DNA_01perDMSO_cell2 01
Finished 20240626_488_EGFP-CAAX_640_SPY650-DNA_01perDMSO_cell2 02
Finished 20240626_488_EGFP-CAAX_640_SPY650-DNA_01perDMSO_cell3 01
Finished 20240626_488_EGFP-CAAX_640_SPY650-DNA_01perDMSO_cell3 02
Finished 20240626_488_EGFP-CAAX_640_SPY650-DNA_01perDMSO_cell3 03
Finished 20240626_488_EGFP-CAAX_640_SPY650-DNA_10uMCK666_cell1 01
Finished 20240626_488_EGFP-CAAX_640_SPY650-DNA_10uMCK666_cell1 02
Finished 20240626_488_EGFP-CAAX_640_SPY650-DNA_10uMCK666_cell2 01
Finished 20240626_488_EGFP-CAAX_640_SPY650-DNA_10uMCK666_cell2 02
Finished 20240626_488_EGFP-CAAX_640_SPY650-DNA_10uMCK666_cell2 03
Finished 20240626_488_EGFP-CAAX_640_SPY650-DNA_10uMCK666_cell3 01
Finished 20240626_488_EGFP-CAAX_640_SPY650-DNA_10uMCK666_cell4 01
Finished 20240702_488_EGFP-CAAX_640_SPY650-DNA_01perDMSO_cell1 01
Finished 20240702_488_EGFP-CAAX_640_SPY650-DNA_01perDMSO_cell1 02
Finished 20240702_488_EGFP-CAAX_640_SPY650-DNA_01perDMSO_cell1 03
Finished 2

Finished 20240709_488_EGFP-CAAX_640_SPY650-DNA_10uMParaNitroBlebbistatin_cell4 02
Finished 20240709_488_EGFP-CAAX_640_SPY650-DNA_10uMParaNitroBlebbistatin_cell4 03
Finished 20240709_488_EGFP-CAAX_640_actin-halotag_01perDMSO_cell1 01
Finished 20240709_488_EGFP-CAAX_640_actin-halotag_01perDMSO_cell1 02
Finished 20240709_488_EGFP-CAAX_640_actin-halotag_01perDMSO_cell2 01
Finished 20240709_488_EGFP-CAAX_640_actin-halotag_01perDMSO_cell2 02
Finished 20240709_488_EGFP-CAAX_640_actin-halotag_01perDMSO_cell2 03
Finished 20240709_488_EGFP-CAAX_640_actin-halotag_01perDMSO_cell2 04
Finished 20240709_488_EGFP-CAAX_640_actin-halotag_01perDMSO_cell3 01
Finished 20240709_488_EGFP-CAAX_640_actin-halotag_01perDMSO_cell3 02
Finished 20240709_488_EGFP-CAAX_640_actin-halotag_01perDMSO_cell3 03
Finished 20240709_488_EGFP-CAAX_640_actin-halotag_01perDMSO_cell3 04
Finished 20240709_488_EGFP-CAAX_640_actin-halotag_01perDMSO_cell3 05
Finished 20240709_488_EGFP-CAAX_640_actin-halotag_01perDMSO_cell3 06
Finished

Finished 20240711_488_EGFP-CAAX_640_actin-halotag_10uMParaNitroBlebbistatin_cell4 07
Finished 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell1 01
Finished 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell1 02
Finished 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell1 03
Finished 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell1 04
Finished 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell1 05
Finished 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell1 06
Finished 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell1 07
Finished 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell1 08
Finished 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell2 01
Finished 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell2 02
Finished 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell2 03
Finished 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell2 04
Finished 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell2 05
Finished 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell2 06
Finished 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell2 07
Finished 20240805_488_EGFP-CAAX_640_SPY650-DNA_cell2 08
Finished 20240805_4

In [3]:
############## GET SURFACE MESHES AND OTHER SHAPE STATS ##############

mindir = 'E:/Aaron/random_lls/'
savedir = mindir + 'processed_images/'
csvdir = mindir + 'processed_data/'
#make dirs if it doesn't exist
datadir = mindir + 'Data_and_Figs/'
meshf = mindir+'Meshes/'  
if not os.path.exists(meshf):
    os.makedirs(meshf)
pilrf = mindir+'PILRs/'
if not os.path.exists(pilrf):
    os.makedirs(pilrf)

    

xyres = 0.145 #um / pixel 
zstep = 0.145 # um
align_method = 'trajectory'
norm_rot = 'provided'
l_order = 10
nisos = [1,63]
pilr_method = 'raw'
sigma = 0
errorlist = []


if norm_rot == 'provided':
    widthpeaks = pd.read_csv(datadir + 'Closest_Width_Peaks.csv', index_col = 0)
    
#get all segmented images that were analyzed
datalist = [x.split('_cell_info.csv')[0] for x in os.listdir(csvdir)]
imlist = [x for x in os.listdir(savedir) if x.endswith('segmented.tiff') and x.split('_segmented.tiff')[0] in datalist]


start = 0
stop = 300
allresults = []
while start<len(imlist):
    print(f'Finished {start}, starting {start}-{stop}')
    if __name__ ==  '__main__':
        results = []
        pool = multiprocessing.Pool(processes=60)
        for i in imlist[start:stop]:
            
            #choose structure name based on file name
            if 'actin' in i:
                str_name = 'actin'
            elif ('Hoechst' in i) or ('DNA' in i):
                str_name = 'nucleus'
            elif 'myosin' in i:
                str_name = 'myosin'
            else:
                str_name = ''
            
            #assign the normal rotation value for that particular cell
            if (norm_rot == 'provided') or (type(norm_rot) == float):
#                 try:
                norm_rot = float(widthpeaks[widthpeaks.cell == i.split('_segment')[0]]['Closest_minimums'].values[0])
#                 #exception for if 
#                 except:
#                     norm_rot = 'widest weighted'
                    
            #put in the pool
            pool.apply_async(shparam_mod.shcoeffs_and_PILR_nonuc, args = (
                i,
                mindir,
                xyres,
                zstep,
                str_name,
                errorlist,
                norm_rot,
                l_order,
                nisos,
                pilr_method,
                sigma,
                align_method,
                ),             
                callback = collect_results)

        pool.close()
        pool.join()
    
    allresults.extend(results)
    
    start = stop + 1
    stop = stop + 1000
    if stop>len(imlist):
        stop = len(imlist)

errorlist = []
bigdf = pd.DataFrame()

for r in allresults:
    

    Shape_Stats = pd.DataFrame([r[0].values()],
                                  columns = list(r[0].keys()))
    cell_coeffs = pd.DataFrame([r[1].values()],
                               columns = list(r[1].keys()))

    bigdf = bigdf.append(pd.concat([Shape_Stats,cell_coeffs], axis=1))

    errorlist.extend(r[2])


#save the shape metrics dataframe
bigdf = bigdf.set_index('cell')
bigdf.to_csv(datadir + 'Shape_Metrics.csv')

#save list of cells that don't have centroid in shape
pd.Series(errorlist).to_csv(datadir + 'ListToExclude.csv')

Finished 0, starting 0-300
Finished 301, starting 301-1300
Finished 1301, starting 1301-2300
Finished 2301, starting 2301-3300
Finished 3301, starting 3301-4300
Finished 4301, starting 4301-5300
Finished 5301, starting 5301-6300
Finished 6301, starting 6301-7300
Finished 7301, starting 7301-8300
Finished 8301, starting 8301-9300
Finished 9301, starting 9301-10300
Finished 10301, starting 10301-11300
Finished 11301, starting 11301-12300
Finished 12301, starting 12301-13300
Finished 13301, starting 13301-14300
Finished 14301, starting 14301-15300
Finished 15301, starting 15301-16300
Finished 16301, starting 16301-17300
Finished 17301, starting 17301-18300
Finished 18301, starting 18301-19300
Finished 19301, starting 19301-20300
Finished 20301, starting 20301-21300
Finished 21301, starting 21301-22300
Finished 22301, starting 22301-23300
Finished 23301, starting 23301-24300
Finished 24301, starting 24301-25300
Finished 25301, starting 25301-26300
Finished 26301, starting 26301-27300
Finis